In [1]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from models import LeNetRegressor
from keras import backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('/home/nicoli/github/alexnet/dataset/csv/imdb_csv/imdb_age_regression.csv', index_col=0)


# In[3]:


cols = list(df.columns[1:])
in_format = list(df.columns)


In [3]:
train_dataset = BatchGenerator(box_output_format=cols)
validation_dataset = BatchGenerator(box_output_format=cols)

train_dataset.parse_csv(labels_filename='/home/nicoli/github/alexnet/dataset/csv/imdb_csv/imdb_age_regression_train_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/alexnet/dataset/imdb-hand-crop/',
                        input_format=in_format)

validation_dataset.parse_csv(labels_filename='/home/nicoli/github/alexnet/dataset/csv/imdb_csv/imdb_age_regression_val_split_47950-70-10-20.csv', 
                             images_dir='/home/nicoli/github/alexnet/dataset/imdb-hand-crop/',
                             input_format=in_format)

In [4]:
img_height, img_width, img_depth = (224,224,3)

epochs = 100

train_batch_size = 64
shuffle = True
ssd_train = False

validation_batch_size = 32

In [5]:

train_generator = train_dataset.generate(batch_size=train_batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         #flip=0.5,
                                         equalize=True,
                                         divide_by_stddev=255,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

validation_generator = validation_dataset.generate(batch_size=validation_batch_size,
                                                   shuffle=shuffle,
                                                   ssd_train=ssd_train,
                                                   #flip=0.5,
                                                   equalize=True,
                                                   divide_by_stddev=255,
                                                   returns={'processed_labels'},
                                                   resize=(img_height, img_width))

print("Number of images in the dataset:", train_dataset.get_n_samples())
print("Number of images in the dataset:", validation_dataset.get_n_samples())


Number of images in the dataset: 33565
Number of images in the dataset: 4795


In [7]:
steps_per_epoch = train_dataset.get_n_samples()/train_batch_size
validation_steps = validation_dataset.get_n_samples()/validation_batch_size

In [10]:
lenet = LeNetRegressor(1, img_width, img_height, img_depth, activation='relu')
lenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 220, 220, 6)       456       
_________________________________________________________________
activation_1 (Activation)    (None, 220, 220, 6)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 6)       0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 106, 106, 16)      2416      
_________________________________________________________________
activation_2 (Activation)    (None, 106, 106, 16)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 44944)             0         
__________

/home/nicoli/github/alexnet/models.py:102: UserWarning: Considering a regression task with output function being relu
  warnings.warn(warning)


In [11]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=True)

In [19]:
csv_logger = CSVLogger('callbacks/lenet/age/history-regression-image-treat-1.csv', append=True, separator=',')

checkpoint = ModelCheckpoint(filepath='callbacks/lenet/age/class-weights-image-treat-1.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=False,
                           period=1)

earlystopping = EarlyStopping(patience=5, mode='min')

callbacks=[checkpoint, csv_logger, earlystopping]

In [13]:
lenet.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [15]:
history = lenet.fit_generator(train_generator, epochs=epochs, 
                             steps_per_epoch=steps_per_epoch, 
                             validation_data=validation_generator,
                             validation_steps=validation_steps)
                             #callbacks=callbacks)

Epoch 1/100
 10/524 [..............................] - ETA: 57:38 - loss: 602.2966 - mean_absolute_error: 19.4193

KeyboardInterrupt: 